# Capitaux 02: AZEC Capitaux Processor Testing

**Purpose**: Test AZEC capital extraction from CAPITXCU and INCENDCU data

**Tests**:
1. Read CAPITXCU and INCENDCU bronze data
2. Extract capital columns (capx_100, capx_cua, smp_sre)
3. Aggregate PE/RD from INCENDCU (mt_baspe, mt_basdi)

---

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))
print(f"Project root: {project_root}")

In [ ]:
from pyspark.sql import SparkSession
from azfr_fsspec_utils import fspath
import azfr_fsspec_abfs

azfr_fsspec_abfs.use()

spark = SparkSession.builder \
    .appName("Capitaux_AZEC_Testing") \
    .getOrCreate()

print(f"✓ Spark {spark.version}")

## 1. Load Configuration

In [ ]:
from utils.loaders.config_loader import ConfigLoader
from src.reader import BronzeReader

config = ConfigLoader(str(project_root / "config" / "config.yml"))
bronze_reader = BronzeReader(
    spark, config,
    str(project_root / "config" / "reading_config.json")
)

VISION = "202509"
print(f"Vision: {VISION}")

## 2. Read CAPITXCU Data

In [ ]:
try:
    df_capitxcu = bronze_reader.read_file_group('capitxcu_azec', VISION)
    print(f"✓ CAPITXCU: {df_capitxcu.count():,} rows")
    print(f"  Columns: {df_capitxcu.columns}")
    df_capitxcu.show(3)
except Exception as e:
    print(f"⚠ CAPITXCU not available: {e}")
    df_capitxcu = None

## 3. Read INCENDCU Data

In [ ]:
try:
    df_incendcu = bronze_reader.read_file_group('incendcu_azec', VISION)
    print(f"✓ INCENDCU: {df_incendcu.count():,} rows")
    print(f"  Columns: {df_incendcu.columns}")
    df_incendcu.show(3)
except Exception as e:
    print(f"⚠ INCENDCU not available: {e}")
    df_incendcu = None

## 4. Test Capital Aggregation (INCENDCU)

In [ ]:
from pyspark.sql.functions import col, sum as spark_sum

if df_incendcu is not None:
    # Aggregate PE and RD by policy
    df_pe_rd = df_incendcu.groupBy('police', 'produit').agg(
        spark_sum('mt_baspe').alias('perte_exp_azec'),
        spark_sum('mt_basdi').alias('risque_direct_azec')
    )
    
    print(f"✓ Aggregated PE/RD by policy: {df_pe_rd.count():,} policies")
    df_pe_rd.show(5)
else:
    print("⚠ Skipping aggregation - no INCENDCU data")

## Summary

In [ ]:
print("="*60)
print("AZEC CAPITAUX TESTING COMPLETE")
print("="*60)
print(f"CAPITXCU: {'Available' if df_capitxcu is not None else 'Not available'}")
print(f"INCENDCU: {'Available' if df_incendcu is not None else 'Not available'}")
print("\n→ Next: Notebook 03 - Consolidation")